In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/aribray/ai-platform-samples/blob/aribray--tabular-notebooks/ai-platform-unified/notebooks/official/tabular-model-training-and-prediction.ipynb"">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/ai-platform-samples/blob/master/ai-platform-unified/notebooks/official/tabular-model-training-and-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>

# AI Platform (Unified) Model Builder SDK: AutoML Tabular Training and Prediction

To use this Colaboratory notebook, you copy the notebook to your own Google Drive and open it with Colaboratory (or Colab). You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Colab automatically displays the return value of the last line in each cell. For more information about running notebooks in Colab, see the Colab welcome page.


## Overview

This tutorial demonstrates how to use the AI Platform (Unified) Python client library to train and deploy a tabular classification model for online prediction.

**Note**: you may incur charges for training, prediction, storage, or usage of other GCP products in connection with testing this SDK.

### Dataset

The dataset we are using is the California Housing Dataset, available locally in Colab. To learn more about this dataset, visit https://developers.google.com/machine-learning/crash-course/california-housing-data-description.

### Objective

This notebook demonstrates how to train and make predictions on an AutoML model based on a tabular dataset. The model is . Alternatively, you can train and make predictions on models by using the `gcloud` command-line tool or by using the online Cloud Console.

The steps performed include the following:

- Create an AI Platform (Unified) model training job.
- Train an AutoML Tabular model.
- Deploy the `Model` resource to a serving `Endpoint` resource.
- Make a prediction by sending data.
- Undeploy the `Model` resource.

### Costs 

This tutorial uses billable components of Google Cloud:

* AI Platform (Unified)
* Cloud Storage

Learn about [AI Platform (Unified)
pricing](https://cloud.google.com/ai-platform-unified/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Set up your local development environment

**If you are using Colab or AI Platform Notebooks**, your environment already meets all the requirements to run this notebook. You can skip this step.

**Otherwise**, make sure your environment meets this notebook's requirements.
You need the following:

* The Google Cloud SDK
* Git
* Python 3
* virtualenv
* Jupyter notebook running in a virtual environment with Python 3

The Google Cloud guide to [Setting up a Python development
environment](https://cloud.google.com/python/setup) and the [Jupyter
installation guide](https://jupyter.org/install) provide detailed instructions
for meeting these requirements. The following steps provide a condensed set of
instructions:

1. [Install and initialize the Cloud SDK.](https://cloud.google.com/sdk/docs/)

1. [Install Python 3.](https://cloud.google.com/python/setup#installing_python)

1. [Install
   virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)
   and create a virtual environment that uses Python 3. Activate the virtual environment.

1. To install Jupyter, run `pip install jupyter` on the
command-line in a terminal shell.

1. To launch Jupyter, run `jupyter notebook` on the command-line in a terminal shell.

1. Open this notebook in the Jupyter Notebook Dashboard.

### Restart the kernel

After you install the additional packages, you need to restart the notebook kernel so it can find the packages.

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### Select a GPU runtime

**Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select "Runtime --> Change runtime type > GPU"**

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

1. [Enable the AI Platform (Unified) API and Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component).

1. If you are running this notebook locally, you will need to install the [Cloud SDK](https://cloud.google.com/sdk).

1. Enter your project ID in the cell below. Then run the cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.

### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [ ]:
PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Otherwise, set your project ID here.

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append it onto the name of resources you create in this tutorial.

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your Google Cloud account

**If you are using AI Platform Notebooks**, your environment is already
authenticated. Skip this step.

**If you are using Colab**, run the cell below and follow the instructions
when prompted to authenticate your account via oAuth.

**Otherwise**, follow these steps:

1. In the Cloud Console, go to the [**Create service account key**
   page](https://console.cloud.google.com/apis/credentials/serviceaccountkey).

2. Click **Create service account**.

3. In the **Service account name** field, enter a name, and
   click **Create**.

4. In the **Grant this service account access to project** section, click the **Role** drop-down list. Type "AI Platform"
into the filter box, and select
   **AI Platform Administrator**. Type "Storage Object Admin" into the filter box, and select **Storage Object Admin**.

5. Click *Create*. A JSON file that contains your key downloads to your
local environment.

6. Enter the path to your service account key as the
`GOOGLE_APPLICATION_CREDENTIALS` variable in the cell below and run the cell.

In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on AI Platform, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This notebook demonstrates how to use Model Builder SDK to create an AutoML model based on a tabular dataset. You will need to provide a Cloud Storage bucket where the dataset will be stored.

Set the name of your Cloud Storage bucket below. It must be unique across all of your 
Cloud Storage buckets.

You may also change the `REGION` variable, which is used for operations
throughout the rest of this notebook. Make sure to [choose a region where AI Platform (Unified) services are
available](https://cloud.google.com/ai-platform-unified/docs/general/locations#available_regions). You may
not use a Multi-Regional Storage bucket for training with AI Platform.

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}
REGION = "us-central1"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [ ]:
! gsutil ls -al $BUCKET_NAME

### Copy dataset into your Cloud Storage bucket

In [ ]:
IMPORT_FILE = 'california-housing.csv'
! gsutil cp cloud-samples-data/ai-platform-unified/datasets/tabular/{IMPORT_FILE} {STAGING_BUCKET}/data

gcs_csv_path = f'{BUCKET_NAME}/data/{IMPORT_FILE}'

### Import AI Platform (Unified) client library

Import the AI Platform (Unified) client library into your Python environment and initialize it.

In [ ]:
import os
import sys

from google.cloud import aiplatform
from google.cloud.aiplatform import gapic as aip

aiplatform.init(project=PROJECT_ID, location=REGION)

## Tutorial

Now you are ready to create your AutoML Tabular model.

### Create a Managed Tabular Dataset from CSV

This section will create a dataset from a CSV file stored on your GCS bucket.

In [ ]:
ds = aiplatform.Dataset.create(
    display_name='california-housing',
    metadata_schema_uri=aiplatform.schema.dataset.metadata.tabular,
    gcs_source=[gcs_csv_path])

ds.resource_name

### Launch a Training Job to Create a Model

Once we have defined your training script, we will create a model. The `run` function creates a training pipeline that trains and creates a `Model` object. After the training pipeline completes, the `run` function returns the `Model` object.

In [ ]:
job = aiplatform.AutoMLTabularTrainingJob(
    display_name='train-housing-automl_1',
    optimization_prediction_type='classification',
    optimization_objective='minimize-rmse',    
    column_transformations=[
        {"numeric": {"column_name": "longitude"}},
        {"numeric": {"column_name": "latitude"}},
        {"numeric": {"column_name": "housing_median_age"}},
        {"numeric": {"column_name": "total_rooms"}},
        {"numeric": {"column_name": "total_bedrooms"}},
        {"numeric": {"column_name": "population"}},
        {"numeric": {"column_name": "households"}},
        {"numeric": {"column_name": "median_income"}},
    ],
    optimization_objective_recall_value=None,
    optimization_objective_precision_value=None,
)

# This will take around an hour to run
model = job.run(
    dataset=ds,
    target_column='median_house_value',
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    weight_column=None,
    budget_milli_node_hours=1000,
    model_display_name='house-value-prediction-model', 
    disable_early_stopping=False,
    predefined_split_column_name=None

### Deploy your model

Before you use your model to make predictions, you need to deploy it to an `Endpoint`. You can do this by calling the `deploy` function on the `Model` resource. This function does two things:

1. Create an `Endpoint` resource to which the `Model` resource will be deployed.
2. Deploy the `Model` resource to the `Endpoint` resource.

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy(machine_type='nl-standard-4')

## Predict on the endpoint

- This samples is taken from an observation where median_house_value = 94600
- Note that the valuse are all strings. Since the original data was in CSV format, everything is treated as a string. The transformations passed into the Dataset inform AI Platform (Unified) to transform the inputs to numeric types.

In [ ]:
prediction = endpoint.predict([
    {"longitude": '-124.35',
     "latitude": '40.54',
     "housing_median_age": '52.0',
     "total_rooms": '1820.0',
     "total_bedrooms": '300.0',
     "population": '806',
     "households": '270.0',
     "median_income": '3.014700', 
    }
])

print(prediction)

## Undeploy the model

To undeploy your `Model` resource from the serving `Endpoint` resource, use the endpoint's `undeploy` method with the following parameter:

- `deployed_model_id`: The model deployment identifier returned by the endpoint service when the `Model` resource is deployed. You can retrieve the deployed models using the endpoint's `deployed_models` property.

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Training Job
- Model
- Endpoint
- Cloud Storage Bucket

In [ ]:
delete_training_job = True
delete_model = True
delete_endpoint = True

# Warning: Setting this to true will delete everything in your bucket
delete_bucket = False

# Delete the training job
job.delete()

# Delete the model
model.delete()

# Delete the endpoint
endpoint.delete()

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil -m rm -r $BUCKET_NAME